<a href="https://colab.research.google.com/github/riteshdurgiani/DLPracs/blob/main/DCPracs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPC

**server**

In [ ]:
import socket

HOST = '127.0.0.1'
PORT = 8080

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))

server.listen(1)
print('Server is listening...')
client_connection, client_address = server.accept()
print('Received connection from client : ', client_address)

while True:
    request = client_connection.recv(1024).decode()
    if request == 'done':
        break
    response = str(eval(request))

    print('Computing ', request)
    print('Result computed : ', response)
    print('Sending result to client...')

    client_connection.send(response.encode())
    print()

client_connection.close()

**client**

In [ ]:
import socket

HOST = '127.0.0.1'
PORT = 8080

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

while True:
    num1 = input('Enter operand 1 : ')
    operator = input('Enter operator (+,-,*,/) : ')
    num2 = input('Enter operand 2 : ')

    request = num1+operator+num2
    client.send(request.encode())

    response = client.recv(1024).decode()
    print('Result is : ', response)

    option = input('Continue? (y/n) : ')
    if option == 'n':
        client.send('done'.encode())
        break
    print()


# Group Communication

**server**

In [ ]:
import socket
import threading

HOST = '127.0.0.1'
PORT = 8080

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(5)
print('Server is listening...')

client_connections = []

def service_client(sender_connection, sender_address):
    while True:
        request = sender_connection.recv(1024).decode()
        print(f'Message received from {sender_address} : ', request)
        print('Sending message to other clients...')
        request = f'Message from {sender_address} : ' + request

        for client_connection in client_connections:
            if client_connection != sender_connection:
                client_connection.send(request.encode())

while True:
    client_connection, client_address = server.accept()
    client_connections.append(client_connection)
    print('Received connection from : ', client_address)

    thread = threading.Thread(target=service_client, args=(client_connection, client_address))
    thread.start()

**client**

In [ ]:
import socket
import threading

HOST = '127.0.0.1'
PORT = 8080

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

def receive_messages():
    while True:
        response = client.recv(1024).decode()
        print(response)

thread = threading.Thread(target=receive_messages)
thread.start()

while True:
    request = input()
    client.send(request.encode())

# Distributed Global Averaging 

In [8]:
import datetime
n = int(input("Enter the number of nodes"))

current_time = []

for i in range(n):
  time = input(f"Enter time in hh:mm format of node {i}")
  time = datetime.datetime.strptime(time,"%H:%M")
  current_time.append((i,time))

agreed_time = input("Enter agreed =upon time in hh:mm format")
agreed_time = datetime.datetime.strptime(agreed_time,"%H:%M")
current_time = sorted(current_time,key=lambda node_time : node_time[1],reverse = True)

def print_time(current_time):
  for i in range(len(current_time)):
    print(f'Node {current_time[i][0]} = {current_time[i][1].time()}')
def print_skews(current_skews):
  for i in range(len(current_skews)):
    print(f'Node {current_skews[i][0]} = {current_skews[i][1]} seconds')
def adjust_time(current_time,skews):
  print("Skews : ")
  for node,skew in skews.items():
    print(f'Node {node} : {skew}')
  
  avg_skews = {node : sum(skew)/len(skew) for node,skew in skews.items()}
  print()
  print('Average Skews Computed ')
  for node,avg in avg_skews.items():
    print(f'Node {node} : {avg} ',end = '')
    status = 'ahead' if avg >= 0 else 'behind'
    print(f'Node {node} is {abs(avg)} seconds {status}')
    print()

  print("Final Adjusted Time : ")
  current_time = [(node_time[0],node_time[1] - datetime.timedelta(seconds=avg_skews[node_time[0]]))
                    for node_time in current_time]
  print_time(current_time)
def DistributedGlobalAveraging(current_time):
  # set the skews 
  skews = dict()
  for j in range(len(current_time)):
    skews[j] = []
  
  #calculate 

  for i in range(len(current_time)):
    current_sender = current_time[i][0]
    #get time to resync 
    time_to_resync = agreed_time - current_time[i][1]
    current_time = [(node_time[0],time_to_resync + node_time[1]) for node_time in current_time]
    print()
    print(f'After {time_to_resync}---')
    current_skews = [(node_time[0],int((node_time[1] - agreed_time).total_seconds())) for node_time in current_time]

    print("Current Time : ")
    print_time(current_time)

    print("Current Skews : ")
    print_skews(current_skews)

    print('Current Sender who broadcast resync : ','Node',current_sender)
    for node,skew in current_skews:
      skews[node].append(skew)
  print()
  adjust_time(current_time,skews)

print("Initital Time ")
print_time(current_time)
print(f'Agreed Upin Time  for resync = {agreed_time.time()}')
DistributedGlobalAveraging(current_time)

Enter the number of nodes3
Enter time in hh:mm format of node 001:00
Enter time in hh:mm format of node 101:10
Enter time in hh:mm format of node 201:30
Enter agreed =upon time in hh:mm format01:15
Initital Time 
Node 2 = 01:30:00
Node 1 = 01:10:00
Node 0 = 01:00:00
Agreed Upin Time  for resync = 01:15:00

After -1 day, 23:45:00---
Current Time : 
Node 2 = 01:15:00
Node 1 = 00:55:00
Node 0 = 00:45:00
Current Skews : 
Node 2 = 0 seconds
Node 1 = -1200 seconds
Node 0 = -1800 seconds
Current Sender who broadcast resync :  Node 2

After 0:20:00---
Current Time : 
Node 2 = 01:35:00
Node 1 = 01:15:00
Node 0 = 01:05:00
Current Skews : 
Node 2 = 1200 seconds
Node 1 = 0 seconds
Node 0 = -600 seconds
Current Sender who broadcast resync :  Node 1

After 0:10:00---
Current Time : 
Node 2 = 01:45:00
Node 1 = 01:25:00
Node 0 = 01:15:00
Current Skews : 
Node 2 = 1800 seconds
Node 1 = 600 seconds
Node 0 = 0 seconds
Current Sender who broadcast resync :  Node 0

Skews : 
Node 0 : [-1800, -600, 0]
Node 

# Berkleys 


In [27]:
import sys

TIME_SERVER_NODE = 0
TIME_SERVER_OFFSET = 0
class Node:
  def __init__(self,id,offset):
    self.id = id
    self.offset = offset
    self.clock = int((datetime.datetime.utcnow() - datetime.datetime(1970, 1, 1)).total_seconds() * 1000) /1000
  def adjust_clock(self,offset):
    self.clock = self.clock + offset
def synchronizeClock(nodes):
  server_time = nodes[TIME_SERVER_NODE].clock +TIME_SERVER_OFFSET
  for node in nodes:
    if node.id != TIME_SERVER_NODE:
      node_time = node.clock + node.offset
      offset = server_time - node_time
      node.adjust_clock(offset)
def run_this():
  nodes = []
  nodes.append(Node(1,10))
  nodes.append(Node(2,-5))
  nodes.append(Node(3,20))
  print("BEFORE")
  for node in nodes:
    print(f'Node {node.id} clock : {node.clock}')
  synchronizeClock(nodes)
  print("After")
  for node in nodes:
    print(f'Node {node.id} clock : {node.clock}')
run_this()

#Looking at the output, we can see that the initial clock values of all nodes are the same because they were initialized with the current time in seconds, and they were created almost simultaneously. After synchronization, the clocks of all nodes have been adjusted to be closer to the time server's clock. Node 1's clock has been adjusted by -10 seconds, Node 2's clock has been adjusted by +5 seconds, and Node 3's clock has been adjusted by -20 seconds, which corresponds to their respective initial offsets from the time server's clock.
#Overall, the algorithm aims to synchronize all the nodes' clocks with the time server's clock, even if they are running on different machines and have different initial offsets from real-time.



BEFORE
Node 1 clock : 1682611387.083
Node 2 clock : 1682611387.083
Node 3 clock : 1682611387.083
After
Node 1 clock : 1682611377.083
Node 2 clock : 1682611392.083
Node 3 clock : 1682611367.083


**ORRRRRRRRRRRRRRR**

In [ ]:
import socket

HOST = '127.0.0.1'
PORT = 8080

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

msg = client.recv(1024).decode()
print(msg)
client_time = input('Enter current time : ')
client.send(client_time.encode())

response = client.recv(1024).decode()
print(response)

In [ ]:
import threading
import socket
import datetime

server_time = input('Enter current time of time server : ')
server_time = datetime.datetime.strptime(server_time, "%H:%M")

HOST = '127.0.0.1'
PORT = 8080

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))

server.listen(4)
print('Time Server is listening...')

client_connections = []
client_time_values = []
skews = []


def service_client(client_connection, client_address, server_time):
    client_connection.send('Server Message : What is your current time (in HH:MM) ? '.encode())

    for client_connection in client_connections:
        client_time = client_connection.recv(1024).decode()
        if client_time == '':
            client_connections.remove(client_connection)
            client_connection.close()
            return
        client_time_values.append((client_connection, client_time))

    print()
    print('Client Clock Values Received :-')
    for client_connection, client_time in client_time_values:
        print(f'Client {client_connection.getpeername()} Clock : {client_time}')
        client_time = datetime.datetime.strptime(client_time, "%H:%M")
        skews.append((client_connection, int((client_time - server_time).total_seconds())))

    print()
    print('Skews Computed :-')
    avg_skew = 0
    for client_connection, skew in skews:
        print(f'Client {client_connection.getpeername()} : {skew} seconds')
        avg_skew += skew

    print()
    avg_skew /= (len(skews)+1)
    print(f'Average Skew by which to adjust server time : {avg_skew} seconds')
    server_time = server_time + datetime.timedelta(seconds=avg_skew)
    print('Adjusted Server Time : ', server_time.time())

    print()
    for client_connection, client_time in client_time_values:
        client_time = datetime.datetime.strptime(client_time, "%H:%M")
        adjustment = int((server_time - client_time).total_seconds())
        print(f'Sending Client {client_connection.getpeername()} Adjustment : {adjustment} seconds')
        msg = f'Server Message : Please adjust your clock by {adjustment} seconds...\nServer Message : Your clock value should be : {server_time.time()}'
        client_connection.send(msg.encode())


while True:
    client_connection, client_address = server.accept()
    client_connections.append(client_connection)
    print('Received connection from : ', client_address)

    thread = threading.Thread(target=service_client, args=(client_connection, client_address, server_time))
    thread.start()

# Bully Algo

In [30]:
n = int(input("Enter the total numver of nodes"))
fault = int(input("enter the faulty node"))
detect = int(input("Enter the detector of failure "))
def find_coordinator(detect,fault,n):
  for node in range(detect,n+1):
    if node == fault:
      continue
    print()
    print(f'----Node {node} sending the election message -----')
    oks = 0
    for nbr in range(node+1,n+1):
      if nbr != fault:
        oks += 1
      print(f'Election message sent to Node {nbr}')
    if oks  > 0:
      print(f'{oks} OKs recieved by Node {node}')
    else:
      print('Active Higher Priority Process Does Not Exist')
      return node
def bully(detect,fault,n):
  print(f"Coordinaor node {fault} has failed")
  print(f'Node {detect} has  detected the failure')
  print(f'Node {detect} initiating election ')

  new_coordinator = find_coordinator(detect,fault,n)

  print()
  print("Result of Election")
  print(f'Node {new_coordinator} elected as new corrdinator')
  print(f'Node {new_coordinator} sending messgae to inform that it is the coordinator')

  for neighbor in range(1,new_coordinator):
    print(f'message sent to {neighbor}')
bully(detect,fault,n)

Enter the total numver of nodes7
enter the faulty node7
Enter the detector of failure 4
Coordinaor node 7 has failed
Node 4 has  detected the failure
Node 4 initiating election 

----Node 4 sending the election message -----
Election message sent to Node 5
Election message sent to Node 6
Election message sent to Node 7
2 OKs recieved by Node 4

----Node 5 sending the election message -----
Election message sent to Node 6
Election message sent to Node 7
1 OKs recieved by Node 5

----Node 6 sending the election message -----
Election message sent to Node 7
Active Higher Priority Process Does Not Exist

Result of Election
Node 6 elected as new corrdinator
Node 6 sending messgae to inform that it is the coordinator
message sent to 1
message sent to 2
message sent to 3
message sent to 4
message sent to 5


# Raymond tree 

In [35]:
class Node:
  def __init__(self,id,parent = None):
    self.id = id
    self.holder = parent
    self.request_queue = []
  def __repr__(self):
    return f"Node {self.id}"
def print_request_queue():
  print("Request Queue : ")
  for i,node in enumerate(nodes):
    print(f'Node {i} : {node.request_queue}')
  print()
def print_holder():
  print('Holder Variables ')
  for i,node in  enumerate(nodes):
    print(f'Node {i} -> Node {node.holder.id}')
def release_cs(current_node):
    print(f'**** Node {current_node.id} is releasing CS ****')

    if len(current_node.request_queue) != 0 :
        requesting_node_id = current_node.request_q[0]
        current_node.request_q.pop(0)
        print(f'Node {current_node.id} sends TOKEN to Node {requesting_node_id}')
        current_node.holder = nodes[requesting_node_id]
        print_request_queue()
        send_token(current_node.holder)

        if len(current_node.request_q) != 0 :
            print(f'Node {current_node.id} sends REQUEST message to Node {current_node.holder.id}')
            request_cs(current_node.holder, current_node)

    else:
        print(f'Node {current_node.id} has released CS and currently holds the TOKEN...')
        print_holder()
        print_request_queue()

def send_token(current_node):
  requesting_node_id = current_node.request_queue[0]
  current_node.request_queue.pop(0)

  if requesting_node_id == current_node.id:
    print(f'**** Node {current_node.id} is executing CS ****')
    current_node.holder = current_node
    print_request_queue()
    release_cs(current_node)
    return 
  print(f'**** Node {current_node.id} sends TOKEN to Node {requesting_node_id} ****')
  current_node.holder = nodes[requesting_node_id]
  print_request_queue()
  send_token(current_node.holder)
def request_cs(current_node,requesting_node):
  if current_node.id != root_id:
    current_node.request_queue.append(requesting_node.id)
    print_request_queue()
    print(f'***Node {current_node.id} sends REQUEST messsage to Node {current_node.holder.id}')
    request_cs(current_node.holder,current_node)
  else:
    if current_node == requesting_node:
      print(f'**** Node {current_node.id} already has the token and so it can execute CS ****')
      return
    current_node.request_queue.append(requesting_node.id)
    print_request_queue()
    current_node.request_queue.pop()
    print(f'***Node {current_node.id} sends REQUEST messsage to Node {requesting_node.id}')
    current_node.holder = requesting_node 
    send_token(current_node.holder)
n = int(input('Enter the number of processs '))
nodes = []
root_id = 0
node_0 = Node(0)
node_0.holder = node_0
nodes.append(node_0)

for i in range(1,n):
  parent_id = int(input(f'Enter the parent id for node {i} : '))
  node = Node(i,nodes[parent_id])
  nodes.append(node)
request_id = int(input('Enter id of node requesting CS : '))
print()
print(f'**** Node {request_id} wants to enter CS ****')
print_holder()
request_cs(nodes[request_id], nodes[request_id])

Enter the number of processs 4
Enter the parent id for node 1 : 0
Enter the parent id for node 2 : 0
Enter the parent id for node 3 : 1
Enter id of node requesting CS : 3

**** Node 3 wants to enter CS ****
Holder Variables 
Node 0 -> Node 0
Node 1 -> Node 0
Node 2 -> Node 0
Node 3 -> Node 1
Request Queue : 
Node 0 : []
Node 1 : []
Node 2 : []
Node 3 : [3]

***Node 3 sends REQUEST messsage to Node 1
Request Queue : 
Node 0 : []
Node 1 : [3]
Node 2 : []
Node 3 : [3]

***Node 1 sends REQUEST messsage to Node 0
Request Queue : 
Node 0 : [1]
Node 1 : [3]
Node 2 : []
Node 3 : [3]

***Node 0 sends REQUEST messsage to Node 1
**** Node 1 sends TOKEN to Node 3 ****
Request Queue : 
Node 0 : []
Node 1 : []
Node 2 : []
Node 3 : [3]

**** Node 3 is executing CS ****
Request Queue : 
Node 0 : []
Node 1 : []
Node 2 : []
Node 3 : []

**** Node 3 is releasing CS ****
Node 3 has released CS and currently holds the TOKEN...
Holder Variables 
Node 0 -> Node 1
Node 1 -> Node 3
Node 2 -> Node 0
Node 3 -> N

# Rickart Agrawala 

In [38]:
import datetime
n = int(input("Enter the number  of sites"))
deffered = {i:[] for i in range(n)}
cs_available = True
requesting = {i: [False,'-'] for i in range(n)}
executing = {i : False for i in range(n)}
replies = {i:0 for i in range(n)}
def request_cs(site):
  if replies[site] > 0:
    print(f'Site S{site} has already requested for CS')
    return
  timestamp = input('Enter ts in HH:MM format ')
  timestamp = datetime.datetime.strptime(timestamp,'%H:%M').time()
  print(f'Site S{site} request  for  CS at timestamp {timestamp}')
  requesting[site] = [True,timestamp]

  for i in range(n):
    if i == site:
      continue
    print(f'S{site} has sent REQUEST message to S{i}')

    if executing[i]:
      print(f'S{i} is currently executing CS...')
      print(f'S{i} has DEFERRED S{site} REQUEST')
      deffered[i].append(site)
    
    elif requesting[i][0]:
      print(f'S{i} has also REQUESTED for CS at timestamp : {requesting[i][1]}')
      if requesting[i][1] > timestamp:
        print(f'S{i} sends REPLY message to S{site}...')
        replies[site] += 1
      else:
        print(f'S{i} DEFERRED S{site} REQUEST')
        deffered[i].append(site)
    else:
      print(f'S{i} sends REPLY message to S{site}...')
      replies[site] += 1
def execute_cs(site):
  if executing[site]:
    print(f'S{site} is already executing CS..')
    print()
    return
  if replies[site] != n-1:
    print(f'S{site} received only {replies[site]} replies so cannot execute CS yet...')
    print()
    return
  global cs_available

  if not cs_available:
    print("CS not available ")
    return 
  print(f'S{site} received all {replies[site]} replies and can now execute CS !!')
  cs_available = False
  executing[site] = True
  print(f'S{site} is executing CS...')
  print()
def release_cs(site):
  if not executing[site]:
    print(f'Site S{site} cannot release CS because it is not executing CS yet..')
    print()
    return
  global cs_available

  cs_available = True
  executing[site] = False
  replies[site] = 0
  requesting[site] = [False,'-']

  for i in range(len(deffered[site])):
    print(f'S{site} is sending REPLY message to S{deffered[site][i]}..')
    replies[deffered[site][i]] += 1
  
  deferred[site] = []
  print()
while True:
    site = int(input(f'Select a site (from 0 to {n-1}) :'))
    print(f'What does S{site} want to do ? ')
    print('1. Request CS')
    print('2. Execute CS')
    print('3. Release CS')
    print('4. Quit')
    choice = int(input('Enter choice : '))
    print()

    if choice == 1:
        request_cs(site)
    elif choice == 2:
        execute_cs(site)
    elif choice == 3:
        release_cs(site)
    else:
        break

Enter the number  of sites3
Select a site (from 0 to 2) :1
What does S1 want to do ? 
1. Request CS
2. Execute CS
3. Release CS
4. Quit
Enter choice : 1

Enter ts in HH:MM format 01:10
Site S1 request  for  CS at timestamp 01:10:00
S1 has sent REQUEST message to S0
S0 sends REPLY message to S1...
S1 has sent REQUEST message to S2
S2 sends REPLY message to S1...
Select a site (from 0 to 2) :2
What does S2 want to do ? 
1. Request CS
2. Execute CS
3. Release CS
4. Quit
Enter choice : 1

Enter ts in HH:MM format 01:09
Site S2 request  for  CS at timestamp 01:09:00
S2 has sent REQUEST message to S0
S0 sends REPLY message to S2...
S2 has sent REQUEST message to S1
S1 has also REQUESTED for CS at timestamp : 01:10:00
S1 sends REPLY message to S2...
Select a site (from 0 to 2) :1
What does S1 want to do ? 
1. Request CS
2. Execute CS
3. Release CS
4. Quit
Enter choice : 2

S1 received all 2 replies and can now execute CS !!
S1 is executing CS...

Select a site (from 0 to 2) :0
What does S0 wa

# Bankers


In [39]:
n = int(input("Enter number of processes : "))
r = int(input("Enter number of resources : "))
alloc = [[0] * r for _ in range(n)]
mx = [[0] * r for _ in range(n)]
need = [[0] * r for _ in range(n)]
avail = [0] * r
safe = []

for i in range(r):
    avail[i] = int(input(f"Enter total number of available instances of resource {i+1} : "))
print()

for i in range(n):
    print(f"Enter information for Process {i+1} : ")
    for j in range(r):
        mx[i][j] = int(input(f"Enter max demand of instances of resource {j+1} for Process {i+1} : "))
        need[i][j] = mx[i][j]
        alloc[i][j] = int(input(f"Enter number of instances of resource {j+1} allocated to Process {i+1} : "))
        need[i][j] -= alloc[i][j]
        avail[j] -= alloc[i][j]
    print()

def display(mx, alloc, need, avail):
    print('Available Resources : ',avail)
    print('Allocated Matrix :- ')
    print(alloc)
    print()
    print('Max Matrix :- ')
    print(mx)
    print()
    print('Need Matrix :- ')
    print(need)
    print()

def bankers(n,r,mx,alloc,need,avail):
    print('***** Initially *****')
    display(mx,alloc,need,avail)

    while True:
        mark = True

        for i in range(n):
            if i+1 in safe:
                continue

            mark = True
            for j in range(r):
                if need[i][j] > avail[j]:
                    mark = False
                    break

            if mark:
                safe.append(i+1)
                for j in range(r):
                    avail[j] += alloc[i][j]
                    alloc[i][j] = 0
                    need[i][j] = '-'

                print(f'***** After allocating resources to P{i+1} *****')
                print(f'P{i + 1} can be allocated resources for execution..')
                display(mx,alloc,need,avail)
                print()
                break

        if not mark:
            print("System is NOT in safe state !!")
            break
        if len(safe) == n:
            print("System is in safe state !!")
            print("Safe Sequence is : ", end=" ")
            for i in safe:
                print(f"P{i}", end=" ")
            print()
            break

bankers(n,r,mx,alloc,need,avail)
3


Enter number of processes : 3
Enter number of resources : 3
Enter total number of available instances of resource 1 : 6
Enter total number of available instances of resource 2 : 6
Enter total number of available instances of resource 3 : 6

Enter information for Process 1 : 
Enter max demand of instances of resource 1 for Process 1 : 4
Enter number of instances of resource 1 allocated to Process 1 : 2
Enter max demand of instances of resource 2 for Process 1 : 3
Enter number of instances of resource 2 allocated to Process 1 : 2
Enter max demand of instances of resource 3 for Process 1 : 6
Enter number of instances of resource 3 allocated to Process 1 : 1

Enter information for Process 2 : 
Enter max demand of instances of resource 1 for Process 2 : 2
Enter number of instances of resource 1 allocated to Process 2 : 3
Enter max demand of instances of resource 2 for Process 2 : 2
Enter number of instances of resource 2 allocated to Process 2 : 2
Enter max demand of instances of resource 3

# Load Balancing 

In [40]:
def print_load(load):
    print()
    print('***** Current Load of System *****')
    for i in range(len(load)):
        print(f'Server {i+1} load : {load[i]} processes')
    print()

def balance_load(n,p):
    load = [0]*n
    for i in range(p):
        load[i%len(load)] += 1
    return load

n = int(input('Enter number of servers : '))
p = int(input('Enter number of processes : '))

while True:
    load = balance_load(n,p)
    print_load(load)
    print('1. Add Servers')
    print('2. Add Processes')
    print('3. Remove Servers')
    print('4. Remove Processes')
    print('5. Exit')
    opt = int(input('Select an option : '))

    if opt == 1:
        change = int(input('Enter number of servers to add : '))
        n += change
    if opt == 2:
        change = int(input('Enter number of processes to add : '))
        p += change
    if opt == 3:
        change = int(input('Enter number of servers to remove : '))
        n -= change
    if opt == 4:
        change = int(input('Enter number of processes to remove : '))
        p -= change
    if opt == 5:
        break


Enter number of servers : 4
Enter number of processes : 7

***** Current Load of System *****
Server 1 load : 2 processes
Server 2 load : 2 processes
Server 3 load : 2 processes
Server 4 load : 1 processes

1. Add Servers
2. Add Processes
3. Remove Servers
4. Remove Processes
5. Exit
Select an option : 4
Enter number of processes to remove : 1

***** Current Load of System *****
Server 1 load : 2 processes
Server 2 load : 2 processes
Server 3 load : 1 processes
Server 4 load : 1 processes

1. Add Servers
2. Add Processes
3. Remove Servers
4. Remove Processes
5. Exit
Select an option : 3
Enter number of servers to remove : 2

***** Current Load of System *****
Server 1 load : 3 processes
Server 2 load : 3 processes

1. Add Servers
2. Add Processes
3. Remove Servers
4. Remove Processes
5. Exit
Select an option : 5


# Multithreading

In [45]:
import threading

lock = threading.Lock()
next_odd = threading.Event()
next_even = threading.Event()

def print_odd(n):
    for num in range(1, n+1, 2):
        with lock:
            print(f'Odd Thread ({threading.current_thread().name}) prints : {num}')
            next_even.set()
            next_odd.clear()
        if num != n:
            next_odd.wait()


def print_even(n):
    for num in range(2, n+1, 2):
        with lock:
            print(f'Even Thread ({threading.current_thread().name}) prints : {num}')
            next_odd.set()
            next_even.clear()
        if num != n:
            next_even.wait()

n = int(input('Enter upper limit : '))

odd_thread = threading.Thread(target=print_odd, args=(n,))
even_thread = threading.Thread(target=print_even, args=(n,))

odd_thread.start()
even_thread.start()

odd_thread.join()
even_thread.join()

print('Done Printing !!')

Enter upper limit : 10
Odd Thread (Thread-18) prints : 1
Even Thread (Thread-19) prints : 2
Odd Thread (Thread-18) prints : 3
Even Thread (Thread-19) prints : 4
Odd Thread (Thread-18) prints : 5
Even Thread (Thread-19) prints : 6
Odd Thread (Thread-18) prints : 7
Even Thread (Thread-19) prints : 8
Odd Thread (Thread-18) prints : 9
Even Thread (Thread-19) prints : 10
Done Printing !!
